In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import json

In [3]:
driver = webdriver.Chrome('F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe')

C:\Users\admin\AppData\Local\Temp/ipykernel_3944/1960677536.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe')


In [106]:
link = 'https://www.goodreads.com/book/show/19288043-gone-girl'
driver.get(link)

In [120]:
rating = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']") # Testing line 
rating.text

"Who are you?\nWhat have we done to each other?\n\nThese are the questions Nick Dunne finds himself asking on the morning of his fifth wedding anniversary when his wife Amy suddenly disappears. The police suspect Nick. Amy's friends reveal that she was afraid of him, that she kept secrets from him. He swears it isn't true. A police examination of his computer shows strange searches. He says they weren't made by him. And then there are the persistent calls on his mobile phone.\n\nSo what did happen to Nick's beautiful wife?"

In [222]:
scraper = BookScraper('https://www.goodreads.com/book/show/19288043-gone-girl')
time.sleep(2)
scraper.get_title()
scraper.get_rating()
scraper.get_author_name()

TypeError: BookScraper() missing 1 required positional argument: 'path'

In [ ]:
#name, author, rating, year, genre, synopsis

In [148]:
class BookScraper:

    def __init__(self, link):
        self.chrome_path = 'F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe'
        self.driver = webdriver.Chrome(self.chrome_path)
        self.driver.get(link)

    def get_title(self):
        self.title = self.driver.find_element(by = By.CSS_SELECTOR, value = "h1[class = 'Text Text__title1']")
        return self.title.text

    def get_rating(self):
        self.rating = self.driver.find_element(by = By.CSS_SELECTOR, value = "div[class = 'RatingStatistics__rating']")
        return float(self.rating.text)
    
    def get_author_name(self):
        self.name = self.driver.find_element(by = By.CSS_SELECTOR, value = "span[class = 'ContributorLink__name']")
        return self.name.text

    def get_synopsis(self):
        try:        
            self.driver.find_element(by = By.CSS_SELECTOR, value = "button[class = 'Button Button--inline Button--small']").click()
        except:
            synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']")
            return synopsis.text
        synopsis = synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']")
        return synopsis.text

In [47]:
def BookScraper(link):
    driver.get(link)
    try:
        driver.find_element(by = By.CSS_SELECTOR, value = "a#rating_details").text
        title = driver.find_element(by = By.CSS_SELECTOR, value = "h1#bookTitle").text
        rating = driver.find_element(by = By.CSS_SELECTOR, value = "span[itemprop = 'ratingValue']").text
        name = driver.find_element(by = By.CSS_SELECTOR, value = "a[class = 'authorName']").text
        num_ratings = driver.find_elements(by = By.CSS_SELECTOR, value  = "a[href = '#other_reviews']")[0].text
        num_reviews = driver.find_elements(by = By.CSS_SELECTOR, value  = "a[href = '#other_reviews']")[1].text
        try:
            driver.find_element(by = By.CSS_SELECTOR, value = "a[onclick = 'swapContent($(this));; return false;']").click()
        except:
            synopsis =  driver.find_element(by = By.CSS_SELECTOR, value= "div[id = 'description']").text        
        else:
            synopsis =  driver.find_element(by = By.CSS_SELECTOR, value= "div[id = 'description']").text
    except:
        title = driver.find_element(by = By.CSS_SELECTOR, value = "h1[class = 'Text Text__title1']").text
        rating = driver.find_element(by = By.CSS_SELECTOR, value = "div[class = 'RatingStatistics__rating']").text
        name = driver.find_element(by = By.CSS_SELECTOR, value = "span[class = 'ContributorLink__name']").text
        num_ratings = driver.find_element(by = By.CSS_SELECTOR, value= "span[data-testid = 'ratingsCount']").text
        num_reviews = driver.find_element(by = By.CSS_SELECTOR, value= "span[data-testid = 'reviewsCount']").text
        try:
            driver.find_element(by = By.CSS_SELECTOR, value = "button[class = 'Button Button--inline Button--small']").click()
        except:
            synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']").text    
        else:
            synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']").text
    return [title, rating, name, num_ratings, num_reviews, synopsis]
    

In [253]:
scraper = BookScraper("https://www.goodreads.com/book/show/2429135.The_Girl_with_the_Dragon_Tattoo")

C:\Users\admin\AppData\Local\Temp/ipykernel_8528/2434887134.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe')


In [254]:
scraper[5]

'Harriet Vanger, a scion of one of Sweden’s wealthiest families disappeared over forty years ago. All these years later, her aged uncle continues to seek the truth. He hires Mikael Blomkvist, a crusading journalist recently trapped by a libel conviction, to investigate. He is aided by the pierced and tattooed punk prodigy Lisbeth Salander. Together they tap into a vein of unfathomable iniquity and astonishing corruption.\n\nAn international publishing sensation, Stieg Larsson’s The Girl with the Dragon Tattoo combines murder mystery, family saga, love story, and financial intrigue into one satisfyingly complex and entertainingly atmospheric novel. (less)'

In [ ]:
df = pd.DataFrame(columns=['title', 'rating', 'name', 'num_ratings', 'num_reviews', 'synopsis'])

In [5]:
genre = 'thriller' 
genrelink = "https://www.goodreads.com/shelf/show/" + genre + "?page=1"

In [5]:
path = 'F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get("https://www.goodreads.com/")

C:\Users\admin\AppData\Local\Temp/ipykernel_11420/4134861425.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [48]:
t = BookScraper("https://www.goodreads.com/book/show/19288043-gone-girl")

In [57]:
t[4][:-8]

'142,390'

In [44]:
s = driver.find_elements(by = By.CSS_SELECTOR, value  = "a[href = '#other_reviews']")
len(s)
s[1].text

'142,390 reviews'

In [24]:
df = pd.DataFrame(columns=['title', 'rating', 'name', 'num_ratings', 'num_reviews', 'synopsis'])

In [25]:
df = df.append({'title' : t[0], 'rating':t[1], 'name':t[2], 'num_ratings':t[3], 'num_reviews':t[4], 'synopsis':t[5]}, ignore_index= True)

In [26]:
df.head()

,title,rating,name,num_ratings,num_reviews,synopsis
0,Gone Girl,4.11,Gillian Flynn,,,Who are you?\nWhat have we done to each other?...


In [11]:
driver.get(genrelink)
books = driver.find_elements(by = By.CSS_SELECTOR, value = "a.bookTitle")
data = []
book_links = []
for i in range(len(books)):
    book_link = books[i].get_attribute('href')
    book_links.append(book_link)
for i in book_links[0:5]:
    out = BookScraper(i)    
    df = df.append({'title' :out[0], 'rating' :out[1], 'name' :out[2], 'num_ratings' :out[3], 'num_reviews' :out[4], 'synopsis' :out[5]}, ignore_index=True)

KeyboardInterrupt: 

In [281]:
len(df)

0

In [258]:
df.append({'name' : 'atharva', 'title' : 'inamdar'}, ignore_index= True)

,name,title
0,atharva,inamdar
